# Decoding through the epoch's window time the sentence / constituent embeddings

We:

- Calculate the embeddings for each sentence / constituent

- Decode it on the epoch window

- Plot it for each condition (level / start)


Todo: 

Integrate this:

```sent_starts = epochs['word_id==0'].apply_baseline((-.300, 0.))
sent_starts.average().plot()

sent_stops = epochs['is_last_word']
bsl = (epochs.times>-.300 )*(epochs.times<=0)
baseline_starts = sent_starts.get_data()[:, :, bsl].mean(-2)

sent_stop_data = sent_stops.get_data()
n_sentences, n_channels, n_times = sent_stop_data.shape
sent_stop_data -= baseline_starts[:, :, None]```


In [3]:
from dataset import read_raw, get_subjects, get_path, add_embeddings
from utils import decod_xy, mne_events
import mne
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from utils import match_list
import spacy

In [4]:
def epoch_add_metadata(modality, subject, levels, starts, runs=9):
    """
    Takes as input subject number, modality, levels of epoching wanted (word, sentence, constituent)
    and starts (onset, offset) as well as the number of total runs (for debugging).
    
    Returns: 
    
    A dict of epochs objects, concatenated on the key (levels x starts)
    
    e.g: {'word_onset': <Epochs 10000 objects>, 'sentence_offset': <Epochs 1000 objects> ....}
    """
    dict_epochs = dict() # DICT containing epochs grouped by conditions (start x level)
    
    # Initialization of the dictionary
    for start in starts: 
            for level in levels:
                epoch_key = f'{level}_{start}'
                dict_epochs[epoch_key] = [] 
                
    # Iterating on runs, building the metadata and re-epoching
    for run in range(1,runs+1):
        raw, meta_, events = read_raw(subject, run, events_return = True, modality=modality)
        meta = meta_.copy()
        
        # Metadata update
        meta['word_onset'] = True
        meta['word_stop'] = meta.start + meta.duration
        meta['sentence_onset'] = meta.word_id == 0
        meta['prev_closing'] = meta['n_closing'].shift(1)
        meta['constituent_onset'] = meta.apply(lambda x: x['prev_closing'] > x['n_closing'] and x['n_closing'] == 1, axis=1)
        meta['constituent_onset'].fillna(False, inplace=True)
        meta.drop('prev_closing', axis=1, inplace=True)
        
        # Adding the sentence stop info
        meta['sentence_id'] = np.cumsum(meta.sentence_onset)
        for s, d in meta.groupby('sentence_id'):
            meta.loc[d.index, 'sent_word_id'] = range(len(d))
            meta.loc[d.index, 'sentence_start'] = d.start.min()
            meta.loc[d.index, 'sentence_stop'] = d.start.max()
            
        # Adding the constituents stop info
        meta['constituent_id'] = np.cumsum(meta.constituent_onset)
        for s, d in meta.groupby('constituent_id'):
            meta.loc[d.index, 'constituent_start'] = d.start.min()
            meta.loc[d.index, 'constituent_stop'] = d.start.max()
            meta.loc[d.index, 'const_word_id'] = range(len(d))
        
        # Adding embeddings info
        meta = add_embeddings(meta, run, 'constituent')
        meta = add_embeddings(meta, run, 'sentence')
        
        embeddings = meta.word.apply(lambda word: nlp(word).vector).values
        meta['embeds_word'] = embeddings
        for start in starts: 
            for level in levels:
                # Select only the rows containing the True for the conditions
                # Simplified to only get for the onset: sentence onset epochs, constituent onset epochs,etc
                sel = meta.query(f'{level}_onset==True')
                assert sel.shape[0] > 10  #

                # Epoching from the metadata having all onset events: if the start=Offset, the mne events
                # Function will epoch on the offset of each level instead of the onset
                # TODO: add adaptative baseline
                epochs = mne.Epochs(raw, **mne_events(sel, raw ,start=start, level=level), decim = 100,
                                     tmin = epoch_windows[f'{level}'][f'{start}_min'],
                                       tmax = epoch_windows[f'{level}'][f'{start}_max'],
                                         event_repeated = 'drop',
                                            preload=True,
                                                baseline=None)
                epoch_key = f'{level}_{start}'

                dict_epochs[epoch_key].append(epochs)
            
    # Once we have the dict of epochs per condition full (epoching for each run for a subject)
    # we can concatenate them, and fix the dev_head             
    for start_ in starts: 
        for level_ in levels:
            epoch_key = f'{level_}_{start_}'
            all_epochs_chosen = dict_epochs[epoch_key]
            # Concatenate epochs

            for epo in all_epochs_chosen:
                epo.info["dev_head_t"] = all_epochs_chosen[1].info["dev_head_t"]

            dict_epochs[epoch_key] = mne.concatenate_epochs(all_epochs_chosen)
            
    return dict_epochs

In [5]:
def decoding_from_criterion(criterion, dict_epochs, starts, levels):
    """
    Input:
    - criterion: the criterion on which the decoding will be done (embeddings, wlength, w_freq, etc..)
    - dict_epochs: the dictionnary containing the epochs for each condition (starts x levels)
    - starts: (onset, offset)
    - levels: (word, sentence, constituent)
    
    Returns:
    Two dataframes: 
    - all_scores: decoding scores for each subject / starts x levels
    - all_evos: decoding scores for each subject / starts x levels
    
    
    """
    # All epochs -> Decoding and generate evoked potentials
    for start in starts: 
        for level in levels:
            epoch_key = f'{level}_{start}'
            epochs = dict_epochs[epoch_key]
            # mean
            evo = epochs.copy().pick_types(meg=True).average(method='median')
            all_evos.append(dict(subject=subject, evo=evo, start=start, level=level))


            # decoding word emb
            epochs = epochs.load_data().pick_types(meg=True, stim=False, misc=False)
            X = epochs.get_data()
            if criterion == 'emb_sentence' or criterion == 'emb_constituent':
                embeddings = epochs.metadata[f'embeds_{level}']
                embeddings = np.vstack(embeddings.values)
                R_vec = decod_xy(X, embeddings)
                scores = np.mean(R_vec, axis=1)
            elif criterion == 'emb_word':
                nlp = spacy.load("fr_core_news_sm")
                embeddings = epochs.metadata.word.apply(lambda word: nlp(word).vector).values
                embeddings = np.array([emb for emb in embeddings])
                R_vec = decod_xy(X, embeddings)
                scores = np.mean(R_vec, axis=1)
            elif criterion == 'word_length':
                y = epochs.metadata.wlength
                R_vec = decod_xy(X, y)
                scores = R_vec

            for t, score in enumerate(scores):
                all_scores.append(dict(subject=subject, score=score, start=start, level=level, t=epochs.times[t]))
    all_scores = pd.DataFrame(all_scores)
    all_evos = pd.DataFrame(all_evos)
    return all_scores, all_evos

In [6]:
modality = "visual"
nlp = spacy.load("fr_core_news_sm")
all_evos = []
all_scores = []
path = get_path(modality)
subjects = get_subjects(path)
runs = 2
decoding_criterion = 'wlength'
epoch_windows = {"word": {"onset_min": -0.3, "onset_max": 1.0, "offset_min": -1.0, "offset_max": 0.3},
                  "constituent": {"offset_min": -2.0, "offset_max": 0.5, "onset_min": -0.5, "onset_max": 2.0},
                  "sentence": {"offset_min": -4.0, "offset_max": 1.0, "onset_min": -1.0, "onset_max": 4.0}}
levels = ('word','constituent','sentence')
starts = ('onset', 'offset')

if isinstance(levels, str):
    levels = [levels]
    
if isinstance(starts, str):
    starts = [starts]
      
# Iterate on subjects to epochs, and mean later
for subject in subjects[2:3]:
    dict_epochs = epoch_add_metadata(modality, subject, levels, starts, runs)
    
    all_scores, all_evos = decoding_from_criterion(decoding_criterion, dict_epochs, starts, levels)

Reading raw files for modality: visual

 Epoching for run 1, subject: 3

Opening raw data file /home/is153802/data/LPP_MEG_visual/sub-3/ses-01/meg/sub-3_ses-01_task-read_run-01_meg.fif...
    Read a total of 13 projection items:
        grad_ssp_upright.fif : PCA-v1 (1 x 306)  idle
        grad_ssp_upright.fif : PCA-v2 (1 x 306)  idle
        grad_ssp_upright.fif : PCA-v3 (1 x 306)  idle
        grad_ssp_upright.fif : PCA-v4 (1 x 306)  idle
        grad_ssp_upright.fif : PCA-v5 (1 x 306)  idle
        mag_ssp_upright.fif : PCA-v1 (1 x 306)  idle
        mag_ssp_upright.fif : PCA-v2 (1 x 306)  idle
        mag_ssp_upright.fif : PCA-v3 (1 x 306)  idle
        mag_ssp_upright.fif : PCA-v4 (1 x 306)  idle
        mag_ssp_upright.fif : PCA-v5 (1 x 306)  idle
        mag_ssp_upright.fif : PCA-v6 (1 x 306)  idle
        mag_ssp_upright.fif : PCA-v7 (1 x 306)  idle
        mag_ssp_upright.fif : PCA-v8 (1 x 306)  idle
    Range : 36000 ... 517999 =     36.000 ...   517.999 secs
Ready.
Reading e

/mnt/localdrive/workspace-LPP/code/neurospin-petit-prince/decoding/local_testing/dataset.py:55: RuntimeWarning: This file contains raw Internal Active Shielding data. It may be distorted. Elekta recommends it be run through MaxFilter to produce reliable results. Consider closing the file and running MaxFilter on the data.
  raw = mne_bids.read_raw_bids(bids_path)
/mnt/localdrive/workspace-LPP/code/neurospin-petit-prince/decoding/local_testing/dataset.py:55: RuntimeWarning: Omitted 81 annotation(s) that were outside data range.
  raw = mne_bids.read_raw_bids(bids_path)
/mnt/localdrive/workspace-LPP/code/neurospin-petit-prince/decoding/local_testing/dataset.py:55: RuntimeWarning: The unit for channel(s) STI001, STI002, STI003, STI004, STI005, STI006, STI007, STI008, STI009, STI010, STI011, STI012, STI013, STI014, STI015, STI016, STI101, STI201, STI301 has changed from V to NA.
  raw = mne_bids.read_raw_bids(bids_path)


Trigger channel has a non-zero initial value of 8 (consider using initial_event=True to detect this event)
Removing orphaned offset at the beginning of the file.
1466 events found
Event IDs: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]
Reading 0 ... 481999  =      0.000 ...   481.999 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 20 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 20.00 Hz
- Upper transition bandwidth: 5.00 Hz (-6 dB cutoff frequency: 22.50 Hz)
- Filter length: 6601 samples (6.601 sec)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 306 out of 306 | elapsed:    6.5s finished


Multiple event values for single event times found. Keeping the first occurrence and dropping all others.
Adding metadata with 31 columns
1412 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 1412 events and 1301 original time points (prior to decimation) ...


...

/tmp/ipykernel_622285/2398031285.py:85: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  dict_epochs[epoch_key] = mne.concatenate_epochs(all_epochs_chosen)
/tmp/ipykernel_622285/2398031285.py:85: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  dict_epochs[epoch_key] = mne.concatenate_epochs(all_epochs_chosen)
/tmp/ipykernel_622285/2398031285.py:85: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  dict_epochs[epoch_key] = mne.concatenate_epochs(all_epochs_chosen)


Adding metadata with 31 columns
251 matching events found
No baseline correction applied
Adding metadata with 31 columns
2973 matching events found
No baseline correction applied
Adding metadata with 31 columns
922 matching events found
No baseline correction applied


/tmp/ipykernel_622285/2398031285.py:85: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  dict_epochs[epoch_key] = mne.concatenate_epochs(all_epochs_chosen)
/tmp/ipykernel_622285/2398031285.py:85: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  dict_epochs[epoch_key] = mne.concatenate_epochs(all_epochs_chosen)
/tmp/ipykernel_622285/2398031285.py:85: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  dict_epochs[epoch_key] = mne.concatenate_epochs(all_epochs_chosen)


Adding metadata with 31 columns
288 matching events found
No baseline correction applied


UnboundLocalError: local variable 'all_evos' referenced before assignment

In [2]:
%debug

ERROR:root:No traceback has been produced, nothing to debug.


In [1]:
from dataset import read_raw, get_subjects, get_path, add_embeddings
from utils import decod_xy, mne_events
import mne
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from utils import match_list
import spacy

modality = "auditory"
nlp = spacy.load("fr_core_news_sm")
all_evos = []
all_scores = []
path = get_path(modality)
subjects = get_subjects(path)
runs = 9
epoch_windows = {"word": {"onset_min": -0.3, "onset_max": 1.0, "offset_min": -1.0, "offset_max": 0.3},
                  "constituent": {"offset_min": -2.0, "offset_max": 0.5, "onset_min": -0.5, "onset_max": 2.0},
                  "sentence": {"offset_min": -4.0, "offset_max": 1.0, "onset_min": -1.0, "onset_max": 4.0}}
levels = ('word','constituent','sentence')
starts = ('onset', 'offset')
      
# Iterate on subjects to epochs, and mean later
for subject in subjects[2:50]:
    
    dict_epochs = dict() # DICT containing epochs grouped by conditions (start x level)
    
    # Initialization of the dictionary
    for start in starts: 
            for level in levels:
                epoch_key = f'{level}_{start}'
                dict_epochs[epoch_key] = [] 
                
    # Iterating on runs, building the metadata and re-epoching
    for run in range(1,runs+1):
        raw, meta_, events = read_raw(subject, run, events_return = True, modality=modality)
        meta = meta_.copy()
        
        # Metadata update
        meta['word_onset'] = True
        meta['word_stop'] = meta.start + meta.duration
        meta['sentence_onset'] = meta.word_id == 0
        meta['prev_closing'] = meta['n_closing'].shift(1)
        meta['constituent_onset'] = meta.apply(lambda x: x['prev_closing'] > x['n_closing'] and x['n_closing'] == 1, axis=1)
        meta['constituent_onset'].fillna(False, inplace=True)
        meta.drop('prev_closing', axis=1, inplace=True)
        
        # Adding the sentence stop info
        meta['sentence_id'] = np.cumsum(meta.sentence_onset)
        for s, d in meta.groupby('sentence_id'):
            meta.loc[d.index, 'sent_word_id'] = range(len(d))
            meta.loc[d.index, 'sentence_start'] = d.start.min()
            meta.loc[d.index, 'sentence_stop'] = d.start.max()
            
        # Adding the constituents stop info
        meta['constituent_id'] = np.cumsum(meta.constituent_onset)
        for s, d in meta.groupby('constituent_id'):
            meta.loc[d.index, 'constituent_start'] = d.start.min()
            meta.loc[d.index, 'constituent_stop'] = d.start.max()
            meta.loc[d.index, 'const_word_id'] = range(len(d))
        
        # Adding embeddings info
        meta = add_embeddings(meta, run, 'constituent')
        meta = add_embeddings(meta, run, 'sentence')
        
        embeddings = meta.word.apply(lambda word: nlp(word).vector).values
        meta['embeds_word'] = embeddings
        for start in starts: 
            for level in levels:
                # Select only the rows containing the True for the conditions
                # Simplified to only get for the onset: sentence onset epochs, constituent onset epochs,etc
                sel = meta.query(f'{level}_onset==True')
                assert sel.shape[0] > 10  #
                
                # Do we need to do that ???
                """
                # Matchlist events and meta
                # So that we can epoch now that's we've sliced our metadata
                if modality == 'auditory':
                    word_events = events[events[:, 2] > 1]
                    meg_delta = np.round(np.diff(word_events[:, 0]/raw.info['sfreq']))
                    meta_delta = np.round(np.diff(sel.onset.values))
                    i, j = match_list(meg_delta, meta_delta)

                # For auditory, we match on the time difference between triggers
                elif modality == "visual":

                    i, j = match_list(events[:, 2], sel.wlength)
                    assert len(i) > (0.9 * len(events))
                    assert (events[i, 2] == sel.loc[j].wlength).mean() > 0.95
                sel = sel.reset_index().loc[j]
                # Making sure there is not hidden bug when matching
                assert sel.shape[0] > 0.5 *  (meta.query(f'{level}_onset==True')).shape[0]
                """
                
                # Epoching from the metadata having all onset events: if the start=Offset, the mne events
                # Function will epoch on the offset of each level instead of the onset
                # TODO: add adaptative baseline
                epochs = mne.Epochs(raw, **mne_events(sel, raw ,start=start, level=level), decim = 100,
                                     tmin = epoch_windows[f'{level}'][f'{start}_min'],
                                       tmax = epoch_windows[f'{level}'][f'{start}_max'],
                                         event_repeated = 'drop',
                                            preload=True,
                                                baseline=None)
                epoch_key = f'{level}_{start}'

                dict_epochs[epoch_key].append(epochs)
            
    # Once we have the dict of epochs per condition full (epoching for each run for a subject)
    # we can concatenate them, and fix the dev_head             
    for start_ in starts: 
        for level_ in levels:
            epoch_key = f'{level_}_{start_}'
            all_epochs_chosen = dict_epochs[epoch_key]
            # Concatenate epochs

            for epo in all_epochs_chosen:
                epo.info["dev_head_t"] = all_epochs_chosen[1].info["dev_head_t"]

            dict_epochs[epoch_key] = mne.concatenate_epochs(all_epochs_chosen)
            
    # All epochs -> Decoding and generate evoked potentials
    for start in starts: 
        for level in levels:
            epoch_key = f'{level}_{start}'
            epochs = dict_epochs[epoch_key]
            # mean
            evo = epochs.copy().pick_types(meg=True).average(method='median')
            all_evos.append(dict(subject=subject, evo=evo, start=start, level=level))


            # decoding word emb
            epochs = epochs.load_data().pick_types(meg=True, stim=False, misc=False)
            X = epochs.get_data()
            embeddings = epochs.metadata[f'embeds_{level}']
            embeddings = np.vstack(embeddings.values)
            #embeddings = epochs.metadata.word.apply(lambda word: nlp(word).vector).values
            #embeddings = np.array([emb for emb in embeddings])
            R_vec = decod_xy(X, embeddings)
            scores = np.mean(R_vec, axis=1)

            for t, score in enumerate(scores):
                all_scores.append(dict(subject=subject, score=score, start=start, level=level, t=epochs.times[t]))

all_scores = pd.DataFrame(all_scores)
all_evos = pd.DataFrame(all_evos)

all_scores.to_csv('./score.csv')
all_evos.to_csv('./evos.csv')

Reading raw files for modality: auditory

 Epoching for run 1, subject: 3

Opening raw data file /home/is153802/data/LPP_MEG_auditory/sub-3/ses-01/meg/sub-3_ses-01_task-listen_run-01_meg.fif...
    Read a total of 13 projection items:
        grad_ssp_upright.fif : PCA-v1 (1 x 306)  idle
        grad_ssp_upright.fif : PCA-v2 (1 x 306)  idle
        grad_ssp_upright.fif : PCA-v3 (1 x 306)  idle
        grad_ssp_upright.fif : PCA-v4 (1 x 306)  idle
        grad_ssp_upright.fif : PCA-v5 (1 x 306)  idle
        mag_ssp_upright.fif : PCA-v1 (1 x 306)  idle
        mag_ssp_upright.fif : PCA-v2 (1 x 306)  idle
        mag_ssp_upright.fif : PCA-v3 (1 x 306)  idle
        mag_ssp_upright.fif : PCA-v4 (1 x 306)  idle
        mag_ssp_upright.fif : PCA-v5 (1 x 306)  idle
        mag_ssp_upright.fif : PCA-v6 (1 x 306)  idle
        mag_ssp_upright.fif : PCA-v7 (1 x 306)  idle
        mag_ssp_upright.fif : PCA-v8 (1 x 306)  idle
    Range : 28000 ... 658999 =     28.000 ...   658.999 secs
Ready.
Rea

/mnt/localdrive/workspace-LPP/code/neurospin-petit-prince/decoding/local_testing/dataset.py:55: RuntimeWarning: This file contains raw Internal Active Shielding data. It may be distorted. Elekta recommends it be run through MaxFilter to produce reliable results. Consider closing the file and running MaxFilter on the data.
  raw = mne_bids.read_raw_bids(bids_path)
/mnt/localdrive/workspace-LPP/code/neurospin-petit-prince/decoding/local_testing/dataset.py:55: RuntimeWarning: The unit for channel(s) STI001, STI002, STI003, STI004, STI005, STI006, STI007, STI008, STI009, STI010, STI011, STI012, STI013, STI014, STI015, STI016, STI101, STI201, STI301 has changed from V to NA.
  raw = mne_bids.read_raw_bids(bids_path)


1954 events found
Event IDs: [  1 129]
Reading 0 ... 630999  =      0.000 ...   630.999 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 20 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 20.00 Hz
- Upper transition bandwidth: 5.00 Hz (-6 dB cutoff frequency: 22.50 Hz)
- Filter length: 6601 samples (6.601 sec)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 306 out of 306 | elapsed:    7.9s finished


Multiple event values for single event times found. Keeping the first occurrence and dropping all others.
Adding metadata with 31 columns
1597 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 1597 events and 1301 original time points (prior to decimation) ...


...

/tmp/ipykernel_593093/1008780087.py:100: RuntimeWarning: The measurement information indicates a low-pass frequency of 20.0 Hz. The decim=100 parameter will result in a sampling frequency of 10.0 Hz, which can cause aliasing artifacts.
  epochs = mne.Epochs(raw, **mne_events(sel, raw ,start=start, level=level), decim = 100,


0 bad epochs dropped
Reading raw files for modality: auditory

 Epoching for run 5, subject: 17

Opening raw data file /home/is153802/data/LPP_MEG_auditory/sub-17/ses-01/meg/sub-17_ses-01_task-listen_run-05_meg.fif...
    Read a total of 13 projection items:
        grad_ssp_upright.fif : PCA-v1 (1 x 306)  idle
        grad_ssp_upright.fif : PCA-v2 (1 x 306)  idle
        grad_ssp_upright.fif : PCA-v3 (1 x 306)  idle
        grad_ssp_upright.fif : PCA-v4 (1 x 306)  idle
        grad_ssp_upright.fif : PCA-v5 (1 x 306)  idle
        mag_ssp_upright.fif : PCA-v1 (1 x 306)  idle
        mag_ssp_upright.fif : PCA-v2 (1 x 306)  idle
        mag_ssp_upright.fif : PCA-v3 (1 x 306)  idle
        mag_ssp_upright.fif : PCA-v4 (1 x 306)  idle
        mag_ssp_upright.fif : PCA-v5 (1 x 306)  idle
        mag_ssp_upright.fif : PCA-v6 (1 x 306)  idle
        mag_ssp_upright.fif : PCA-v7 (1 x 306)  idle
        mag_ssp_upright.fif : PCA-v8 (1 x 306)  idle
    Range : 217000 ... 582999 =    217.000 ... 

/mnt/localdrive/workspace-LPP/code/neurospin-petit-prince/decoding/local_testing/dataset.py:55: RuntimeWarning: This file contains raw Internal Active Shielding data. It may be distorted. Elekta recommends it be run through MaxFilter to produce reliable results. Consider closing the file and running MaxFilter on the data.
  raw = mne_bids.read_raw_bids(bids_path)
/mnt/localdrive/workspace-LPP/code/neurospin-petit-prince/decoding/local_testing/dataset.py:55: RuntimeWarning: Omitted 565 annotation(s) that were outside data range.
  raw = mne_bids.read_raw_bids(bids_path)
/mnt/localdrive/workspace-LPP/code/neurospin-petit-prince/decoding/local_testing/dataset.py:55: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw = mne_bids.read_raw_bids(bids_path)
/mnt/localdrive/workspace-LPP/code/neurospin-petit-prince/decoding/local_testing/dataset.py:55: RuntimeWarning: The unit for channel(s) STI001, STI002, STI003, STI004, STI005, STI006, STI007, STI008, ST

936 events found
Event IDs: [  1 128]


AssertionError: 

In [2]:
dict_epochs

{'word_onset': [<Epochs |  1605 events (all good), -0.3 - 1 sec, baseline off, ~60.8 MB, data loaded, with metadata,
   '1': 1605>,
  <Epochs |  1764 events (all good), -0.3 - 1 sec, baseline off, ~66.3 MB, data loaded, with metadata,
   '1': 1764>,
  <Epochs |  1853 events (all good), -0.3 - 1 sec, baseline off, ~69.4 MB, data loaded, with metadata,
   '1': 1853>,
  <Epochs |  1637 events (all good), -0.3 - 1 sec, baseline off, ~61.9 MB, data loaded, with metadata,
   '1': 1637>],
 'constituent_onset': [<Epochs |  481 events (all good), -0.5 - 2 sec, baseline off, ~36.1 MB, data loaded, with metadata,
   '1': 481>,
  <Epochs |  510 events (all good), -0.5 - 2 sec, baseline off, ~38.0 MB, data loaded, with metadata,
   '1': 510>,
  <Epochs |  565 events (all good), -0.5 - 2 sec, baseline off, ~41.5 MB, data loaded, with metadata,
   '1': 565>,
  <Epochs |  512 events (all good), -0.5 - 2 sec, baseline off, ~38.1 MB, data loaded, with metadata,
   '1': 512>],
 'sentence_onset': [<Epochs

In [ ]:
all_scores = pd.DataFrame(all_scores)
all_evos = pd.DataFrame(all_evos)

from matplotlib.pyplot import figure

figure(figsize=(16, 10), dpi=80)

fig, axes = plt.subplots(3, 2)

for axes_, level in zip( axes, levels):  
    for ax, start in zip( axes_, starts):  
        cond1 = all_scores.level==f'{level}'
        cond2 = all_scores.start==f'{start}'
        data = all_scores[ cond1 & cond2]
        y = []
        x = []
        for s, t in data.groupby('t'):
            score_avg = t.score.mean()
            y.append(score_avg)
            x.append(s)

        ax.plot(x,y)
        ax.set_title(f'{level} {start}')


In [6]:
for i in (tuple("hi")):
    print(i)

h
i
